In [ ]:
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
import os
import dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
loadenv = dotenv.load_dotenv(dotenv_path="./.env")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults()

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant. Think step by step before answering."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [ ]:
llm = ChatOllama(model="llama3.2")

In [ ]:
tools = [search]

In [ ]:
msg_history = ChatMessageHistory()

# Create memory checkpointer for conversation history
memory = MemorySaver()

# Create the agent with tools and memory
agent = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory
)

# Agent executor function for easy invocation
def execute_agent(user_input: str, thread_id: str = "default-session"):
    """
    Execute the agent with a user query.
    
    Args:
        user_input: The user's question or command
        thread_id: Session identifier for conversation continuity
    
    Returns:
        The agent's response
    """
    response = agent.invoke(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": thread_id}}
    )
    
    return response["messages"][-1].content

# Example usage
result = execute_agent("What is the weather in San Francisco?")
print(result)

# Multi-turn conversation example (uses same thread_id for history)
result2 = execute_agent("What about New York?", thread_id="default-session")
print(result2)

## Stream agent responses for real-time output
#def stream_agent(user_input: str, thread_id: str = "default-session"):
#    """
#    Stream the agent's response in real-time.
#    
#    Args:
#        user_input: The user's question or command
#        thread_id: Session identifier for conversation continuity
#    """
#    config = {"configurable": {"thread_id": thread_id}}
#    
#    for chunk in agent.stream(
#        {"messages": [("user", user_input)]},
#        {"configurable": {"thread_id": thread_id}},
#        stream_mode="values"
#    ):
#
## Example streaming
#stream_agent("Tell me about the Eiffel Tower")
